# Load dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('mtsamples.csv')
df.head()

In [ ]:
len(df['description']) # 4999 instances

In [ ]:
# import nltk
# nltk.download()

## Gender

In [ ]:
#extract gender
import nltk
#define dictionary
gender_words = ['male', 'man', 'boy', 'gentleman', 'female', 'woman', 'girl', 'lady', 'Male', 'Female', 'Man', 
                'Woman', 'Boy', 'Girl', 'Gentleman', 'Lady']

final_gender = []

extract_words = []
not_extract_words = []
c = 0

med_spec = list(df['medical_specialty'])
samp_name = list(df['sample_name'])
trans = list(df['transcription'])
key = list(df['keywords'])

for i in list(df['description']):
    flag1 = 0
    flag2 = 0
    new_i = ' ' + i
    for j in gender_words:
        if ' '+j+' ' in new_i or ' '+j+',' in new_i or ','+j+' ' in new_i or ','+j in new_i or ' '+j+'.' in new_i or '.'+j in new_i :
          
            if j in ['male', 'man', 'boy', 'gentleman', 'Male', 'Man', 'Boy', 'Gentleman']: 
                final_gender.append([i, med_spec[c], samp_name[c], trans[c],key[c],'Male'])
                flag2 = 1
            else:
                flag2 = 1
                final_gender.append([i, med_spec[c], samp_name[c], trans[c],key[c], 'Female'])
   
    #use POS tagger
    if flag2!= 1:
        wordsList = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(wordsList)

        for tag_pair in tagged:
            if tag_pair[1] == 'PRP':
                if tag_pair[0] == 'He' or tag_pair[0] == 'he':
                    final_gender.append([i, med_spec[c], samp_name[c], trans[c],key[c], 'Male'])
                    flag2 = 1
                    flag1 = 1
                if tag_pair[0] == 'She' or tag_pair[0] == 'she':
                    flag2 = 1
                    final_gender.append([i, med_spec[c], samp_name[c], trans[c],key[c],  'Female'])
                    flag1 = 1
           
            if tag_pair[1] == 'PRP$':
                if tag_pair[0] == 'Him' or tag_pair[0] == 'him':
                    final_gender.append([i, med_spec[c], samp_name[c], trans[c],key[c],  'Male'])
                    flag2 = 1
                if tag_pair[0] == 'Her' or tag_pair[0] == 'her':
                    flag2 = 1
                    final_gender.append([i, med_spec[c], samp_name[c], trans[c],key[c], 'Female'])
              
                
    if flag2 == 0:
        final_gender.append([i, med_spec[c], samp_name[c], trans[c],key[c], 'none'])
    c = c + 1


In [ ]:
df_final_gender = pd.DataFrame(final_gender, columns = ['Description', 'Medical_specialty', 'Sample_name', 'Transcription', 'Keyword', 'Gender'])
mask = df_final_gender.duplicated(keep='first')

# drop the duplicate rows (Sentence: He is ..., He was then..., 'Male' detected twice, so remove those rows.)
df_final_gender = df_final_gender[~mask]


## Age

In [ ]:
import re
regexes = [
    r'\b\d{1,2}\s*(?:year[s]?|yrs?)[\s-]old\b',  # e.g. "18-years-old", "2-year-old"
    r'\b\d{1,2}\s*-\s*(?:year[s]?|yrs?)-old\b',  # e.g. "2-year-old", "90-years-old"
    r'\b\d{1,2}\s*month[s]?[\s-]old\b',  # e.g. "1 month old", "2 months old"
    r'\b\d{1,2}\s*-\s*month[s]?-old\b',  # e.g. "2-month-old", "11-months-old"
    r'\d{1,2}\s?day(?:s)?\sold',  # e.g. "1 day old", "2 days old"
    r'\d{1,2}-day(?:s)?-old', # e.g. "2-month-old", "11-months-old"
    r'\d{1,2}\s?week(?:s)?\sold', r'\d{1,2}-week(?:s)?-old',
    r'(?<!\ )\b\d+\s*(?:day|week|month|year)s?\s*of\s*age\b', # 2 years of age
    r'middle-aged|middle aged|middle-ages|young age|old age|gestational age',
    r"\b\d{1,3}-years\s+old\b", #1-years old
    r"\b\d{1,3}-year\s+old\b",#1-year old
    r"\b\d{1,3}-months\s+old\b",#1-months old
    r"\b\d{1,3}-month\s+old\b",#1-month old
    r"\b\d{1,3}-weeks\s+old\b",#1-weeks old
    r"\b\d{1,3}-week\s+old\b",#1-week old
    r"\b\d{1,3}-days\s+old\b",#1-days old
    r"\b\d{1,3}-day\s+old\b", #1-day old
    r'\b\d+-years?\b|\b\d+-year\b' 
    r'\b\d+(?:-\d+)?(?:/\d+)?-?(?:year|years)-old?\b',
    r'\b\d+-?\d*/\d+ year(?:s?-old)?'
    
]

null_ages = []
ages = []
final_ages = []
for i in list(df['description']):
    age = []
    for regex in regexes:
        age += re.findall(regex, i)
    if len(age) == 0:
        final_ages.append([i,age])
    else:
        final_ages.append([i,age])


## Treatment

In [ ]:
from transformers import pipeline
qa_model = pipeline("question-answering")

In [ ]:

treatment = []
c=0
question = "What is the treatment administered to the patient?"
for i in list(df['transcription']):
    try:
        ans = qa_model(question = question, context = i)
    except:
        continue
    treatment.append([i,ans['answer']])
